In [15]:
import geopandas as gpd
import pandas as pd

import networkx as nx
from geopy.distance import geodesic  
from tqdm import tqdm
import pickle



# 2. Carregar os dataframes
rotas = pd.read_parquet("dados/rotaaas_historico.parquet")
viagens = pd.read_parquet("dados/viagens_historico.parquet")
estacoes_metro_e_trem = pd.read_parquet("dados/estacoes_metro_e_trem.parquet")
paradas=pd.read_parquet("dados/paradas_historico.parquet")
passageiros=pd.read_parquet("dados\passageiros_unificado.parquet")

In [16]:
rotas.columns

Index(['route_id', 'agency_id', 'route_short_name', 'route_long_name',
       'route_type', 'route_color', 'route_text_color', 'data_referencia'],
      dtype='object')

In [17]:

rotas['data_referencia']=pd.to_datetime(rotas['data_referencia'],format="%Y%m").dt.to_period('M')
viagens['data_referencia']=pd.to_datetime(viagens['data_referencia'],format="%Y%m").dt.to_period('M')
paradas['data_referencia']=pd.to_datetime(paradas['data_referencia'],format="%Y%m").dt.to_period('M')
passageiros['data']=passageiros['data'].dt.to_period('M')

passageiros=passageiros.groupby(["data",'route_id']).sum().reset_index()

In [ ]:
def cria_grafo(data):
    rotas_data=rotas[rotas.data_referencia==data]
    passageiros_data=passageiros[passageiros.data==data]
    viagens_data=viagens[viagens.data_referencia==data]
    paradas_data=paradas[paradas.data_referencia==data]

    mapa_passageiros_data = dict(passageiros_data[['route_id', 'passageiros_total']].values)
    route_info = rotas_data.set_index('route_id')[['route_short_name','route_long_name','route_color','agency_id']].to_dict('index')
    
    # Criar o grafo
    G = nx.Graph()
    # ----------------------------
    # ADICIONAR NÓS DAS PARADAS
    # ----------------------------

    for _, row in paradas_data.iterrows():
        stop_id = row['stop_id']
        lat = row['stop_lat']
        lon = row['stop_lon']
        G.add_node(
            f"parada_{stop_id}",
            tipo='parada',
            latitude=lat,
            longitude=lon,
            bairro=row.get('NM_BAIRRO', None),
            populacao=row.get("v0001",None),
            subprefeituras =row.get("Subprefeituras ",None),
            regiao=row.get("Regiões",None),
            estacao_mais_proxima=row.get('nome_estacao_mais_proxima', None),
            dist_estacao_mais_proxima=row.get('dist_estacao_mais_proxima', None),
            nome_estacao_mais_proxima_existente=row.get('nome_estacao_mais_proxima_existente', None),
            dist_estacao_mais_proxima_existente=row.get('dist_estacao_mais_proxima_existente', None),
            nome_estacao_mais_proxima_projetado=row.get('nome_estacao_mais_proxima_projetado', None),
            dist_estacao_mais_proxima_projetado=row.get('dist_estacao_mais_proxima_projetado', None)
        )

    # ----------------------------
    # ADICIONAR NÓS DAS ESTAÇÕES
    # ----------------------------
    for _, row in estacoes_metro_e_trem.iterrows():
        estacao_id = row['id']
        geometry = row['geometry']
        
        # Extrair lat/lon do campo 'geometry' (formato WKT ou shapely)
        if isinstance(geometry, str) and geometry.startswith('POINT'):
            coords = geometry.replace('POINT (', '').replace(')', '').split()
            lon, lat = map(float, coords)
        else:
            continue  # pula se não conseguir extrair
        
        G.add_node(
            f"estacao_{estacao_id}",
            tipo='estacao',
            existencia=row.get("existencia",None),
            latitude=lat,
            longitude=lon,
            nome=row['nm_estacao_metro_trem'],
            linha=row['nm_linha_metro_trem']
        )

    # ----------------------------
    # ADICIONAR ARESTAS ENTRE PARADAS DE CADA VIAGEM
    # ----------------------------
    print("Adicionando arestas entre paradas...")
    for trip_id, grupo in tqdm(viagens_data.groupby('trip_id')):
        grupo_ordenado = grupo.sort_values(by='stop_sequence')
        stops = grupo_ordenado[['stop_id', 'stop_lat', 'stop_lon']].values
        route_id = grupo_ordenado['route_id'].iloc[0]
        passag = mapa_passageiros_data.get(route_id, None)
        route_short_name=info = route_info[route_id]['route_short_name']
        route_long_name=route_info[route_id]['route_long_name']
        route_color=route_info[route_id]['route_color']
        agency_id=route_info[route_id]['agency_id']

        for i in range(len(stops) - 1):
            stop1 = stops[i]
            stop2 = stops[i + 1]
            
            id1 = f"parada_{int(stop1[0])}"
            id2 = f"parada_{int(stop2[0])}"
            
            coord1 = (stop1[1], stop1[2])
            coord2 = (stop2[1], stop2[2])
            
            dist = geodesic(coord1, coord2).meters

            if not G.has_edge(id1, id2):
                G.add_edge(id1, id2,
                        peso_distancia=dist,
                        peso_passageiros=passag,
                        tipo='onibus',
                        route_id=route_id,
                        route_short_name=route_short_name,
                        route_long_name=route_long_name,
                        route_color=route_color,
                        agency_id=agency_id)
            else:
                if passag is not None:
                    G[id1][id2]['peso_passageiros'] = passag


    print(f"{data} - Grafo final: {G.number_of_nodes()} nós e {G.number_of_edges()} arestas")
    with open(f"dados/grafos/grafo_{data}.pkl", "wb") as f:
        pickle.dump(G, f)
    return G

In [22]:
for data in rotas.data_referencia.drop_duplicates():
    cria_grafo(data)

Adicionando arestas entre paradas...


100%|██████████| 2156/2156 [00:13<00:00, 154.29it/s]


2015-01 - Grafo final: 18794 nós e 25387 arestas
Adicionando arestas entre paradas...


100%|██████████| 2200/2200 [00:13<00:00, 158.98it/s]


2015-02 - Grafo final: 18839 nós e 25475 arestas
Adicionando arestas entre paradas...


...

2025-01 - Grafo final: 21834 nós e 29235 arestas
Adicionando arestas entre paradas...


100%|██████████| 2245/2245 [00:13<00:00, 167.44it/s]


2025-02 - Grafo final: 21849 nós e 29261 arestas
Adicionando arestas entre paradas...


100%|██████████| 2255/2255 [00:13<00:00, 167.75it/s]


2025-03 - Grafo final: 21897 nós e 29320 arestas
